In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import psycopg2
import pandas as pd

In [4]:
DBNAME = "opportunity_youth"

In [5]:
conn = psycopg2.connect(dbname=DBNAME)
cur = conn.cursor()

In [6]:
### checking that all rows are for people, not houses
df = pd.read_sql("SELECT COUNT(*) FROM pums_2017 WHERE rt = 'P' LIMIT 100;", conn)
df

,count
0,359075


In [7]:
### find table names from Database
cur.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
for table in cur.fetchall():
    print(table)

('pums_2017',)
('puma_names_2010',)
('wa_jobs_2017',)
('wa_geo_xwalk',)
('ct_puma_xwalk',)


In [8]:
### find King County puma id's from puma_names
df = pd.read_sql("""
SELECT puma, puma_name
FROM puma_names_2010
WHERE state_fips= '53' AND puma_name LIKE '%King%'
LIMIT 20
""",
           conn)

indexer = 0
for i in df['puma_name']:
    print (df['puma'][indexer])
    print (i)
    indexer += 1

11606
King County (Northwest)--Shoreline, Kenmore & Bothell (South) Cities                                                                                                                                                                                                                                                                                                                                                                                                                                                
11607
King County (Northwest)--Redmond, Kirkland Cities, Inglewood & Finn Hill                                                                                                                                                                                                                                                                                                                                                                                                                               

In [9]:
### find seattle puma id's from puma
df = pd.read_sql("""
SELECT puma, puma_name
FROM puma_names_2010
WHERE puma_name LIKE '%Seattle%'
LIMIT 20
""",
           conn)

indexer = 0
for i in df['puma_name']:
    print (df['puma'][indexer])
    print (i)
    indexer += 1

11601
Seattle City (Northwest)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
11602
Seattle City (Northeast)                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [10]:
### limiting data by South King County
pd.read_sql("""
SELECT COUNT(*)
FROM pums_2017
WHERE puma in ('11610', '11611', '11612', '11613', '11614', '11615')
""",
            conn)

,count
0,34145


In [11]:
### limiting data by SKC and age from 16 - 24, assigning to ay_df
ay_df = pd.read_sql("""
SELECT *
FROM pums_2017
WHERE puma in ('11610', '11611', '11612', '11613', '11614', '11615')
AND agep BETWEEN 16 AND 24
""",
            conn)

ay_df

,rt,serialno,division,sporder,puma,region,st,adjinc,pwgtp,agep,...,pwgtp71,pwgtp72,pwgtp73,pwgtp74,pwgtp75,pwgtp76,pwgtp77,pwgtp78,pwgtp79,pwgtp80
0,P,2013000102767,9,03,11615,4,53,1061971,17.0,22.0,...,27.0,18.0,18.0,16.0,26.0,5.0,5.0,5.0,4.0,17.0
1,P,2013000103566,9,03,11611,4,53,1061971,20.0,17.0,...,37.0,18.0,32.0,4.0,17.0,28.0,40.0,18.0,21.0,19.0
2,P,2013000105303,9,02,11615,4,53,1061971,17.0,23.0,...,28.0,16.0,6.0,5.0,4.0,16.0,28.0,5.0,17.0,17.0
3,P,2013000106284,9,01,11614,4,53,1061971,4.0,23.0,...,4.0,5.0,4.0,6.0,8.0,5.0,5.0,1.0,1.0,3.0
4,P,2013000106963,9,02,11612,4,53,1061971,43.0,19.0,...,15.0,50.0,43.0,41.0,15.0,28.0,41.0,57.0,53.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3477,P,2013000075342,9,04,11611,4,53,1061971,34.0,22.0,...,54.0,15.0,13.0,32.0,48.0,28.0,65.0,31.0,12.0,52.0
3478,P,2013000088597,9,03,11610,4,53,1061971,21.0,17.0,...,19.0,22.0,17.0,24.0,7.0,7.0,7.0,23.0,40.0,39.0
3479,P,2013000088914,9,02,11612,4,53,1061971,36.0,16.0,...,33.0,43.0,69.0,43.0,11.0,63.0,33.0,59.0,38.0,13.0
3480,P,2013000100470,9,03,11613,4,53,1061971,13.0,21.0,...,32.0,3.0,12.0,17.0,12.0,13.0,22.0,19.0,12.0,13.0


In [12]:
### create 3 dataframes based on age grouping of ay_df

ay_df_16_18 =  pd.read_sql("""
SELECT *
FROM pums_2017
WHERE puma in ('11610', '11611', '11612', '11613', '11614', '11615')
AND agep BETWEEN 16 AND 18
""",
            conn)

ay_df_19_21 =  pd.read_sql("""
SELECT *
FROM pums_2017
WHERE puma in ('11610', '11611', '11612', '11613', '11614', '11615')
AND agep BETWEEN 19 AND 21
""",
            conn)

ay_df_22_24 =  pd.read_sql("""
SELECT *
FROM pums_2017
WHERE puma in ('11610', '11611', '11612', '11613', '11614', '11615')
AND agep BETWEEN 22 AND 24
""",
            conn)

In [13]:
ay_df_16_18

,rt,serialno,division,sporder,puma,region,st,adjinc,pwgtp,agep,...,pwgtp71,pwgtp72,pwgtp73,pwgtp74,pwgtp75,pwgtp76,pwgtp77,pwgtp78,pwgtp79,pwgtp80
0,P,2013000103566,9,03,11611,4,53,1061971,20.0,17.0,...,37.0,18.0,32.0,4.0,17.0,28.0,40.0,18.0,21.0,19.0
1,P,2013000106963,9,03,11612,4,53,1061971,24.0,16.0,...,7.0,25.0,38.0,23.0,7.0,25.0,31.0,19.0,34.0,9.0
2,P,2013000115983,9,03,11614,4,53,1061971,4.0,16.0,...,4.0,2.0,4.0,5.0,6.0,8.0,0.0,1.0,7.0,8.0
3,P,2013000121584,9,02,11611,4,53,1061971,19.0,18.0,...,17.0,21.0,32.0,17.0,5.0,12.0,7.0,4.0,30.0,21.0
4,P,2013000121584,9,03,11611,4,53,1061971,18.0,18.0,...,17.0,21.0,32.0,18.0,5.0,11.0,7.0,3.0,30.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,P,2013000069409,9,04,11614,4,53,1061971,9.0,17.0,...,3.0,6.0,8.0,7.0,2.0,14.0,2.0,9.0,14.0,2.0
1309,P,2013000072428,9,03,11615,4,53,1061971,13.0,17.0,...,23.0,3.0,3.0,14.0,15.0,13.0,21.0,14.0,4.0,15.0
1310,P,2013000088914,9,02,11612,4,53,1061971,36.0,16.0,...,33.0,43.0,69.0,43.0,11.0,63.0,33.0,59.0,38.0,13.0
1311,P,2013000088597,9,03,11610,4,53,1061971,21.0,17.0,...,19.0,22.0,17.0,24.0,7.0,7.0,7.0,23.0,40.0,39.0


In [14]:
### limiting data by unemployed status & school status, assigning to oy_df
oy_df = pd.read_sql("""
SELECT *
FROM pums_2017
WHERE puma in ('11610', '11611', '11612', '11613', '11614', '11615')
AND agep BETWEEN 16 AND 24
AND esr != '1'
AND esr != '4'
AND sch = '1'
""",
            conn)

oy_df

,rt,serialno,division,sporder,puma,region,st,adjinc,pwgtp,agep,...,pwgtp71,pwgtp72,pwgtp73,pwgtp74,pwgtp75,pwgtp76,pwgtp77,pwgtp78,pwgtp79,pwgtp80
0,P,2013000106284,9,01,11614,4,53,1061971,4.0,23.0,...,4.0,5.0,4.0,6.0,8.0,5.0,5.0,1.0,1.0,3.0
1,P,2013000303198,9,04,11613,4,53,1061971,10.0,16.0,...,12.0,20.0,3.0,10.0,10.0,16.0,9.0,21.0,20.0,11.0
2,P,2013000111689,9,05,11610,4,53,1061971,40.0,19.0,...,61.0,12.0,51.0,12.0,13.0,41.0,37.0,52.0,32.0,36.0
3,P,2013000118713,9,03,11613,4,53,1061971,25.0,23.0,...,26.0,25.0,29.0,6.0,24.0,26.0,25.0,41.0,24.0,43.0
4,P,2013000121584,9,02,11611,4,53,1061971,19.0,18.0,...,17.0,21.0,32.0,17.0,5.0,12.0,7.0,4.0,30.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,P,2013000058010,9,02,11614,4,53,1061971,45.0,17.0,...,20.0,56.0,51.0,39.0,40.0,52.0,17.0,48.0,51.0,54.0
406,P,2013000059060,9,02,11610,4,53,1061971,19.0,22.0,...,7.0,20.0,16.0,5.0,21.0,5.0,20.0,15.0,17.0,5.0
407,P,2013000067168,9,03,11615,4,53,1061971,22.0,24.0,...,7.0,20.0,24.0,23.0,41.0,7.0,24.0,24.0,33.0,19.0
408,P,2013000100470,9,04,11613,4,53,1061971,16.0,18.0,...,23.0,3.0,16.0,17.0,11.0,12.0,32.0,29.0,20.0,11.0


In [15]:
### create 3 dataframes based on age grouping of oy_df

oy_df_16_18 = pd.read_sql("""
SELECT *
FROM pums_2017
WHERE puma in ('11610', '11611', '11612', '11613', '11614', '11615')
AND agep BETWEEN 16 AND 18
AND esr != '1'
AND esr != '4'
AND sch = '1'
""",
            conn)

oy_df_19_21 = pd.read_sql("""
SELECT *
FROM pums_2017
WHERE puma in ('11610', '11611', '11612', '11613', '11614', '11615')
AND agep BETWEEN 19 AND 21
AND esr != '1'
AND esr != '4'
AND sch = '1'
""",
            conn)

oy_df_22_24 = pd.read_sql("""
SELECT *
FROM pums_2017
WHERE puma in ('11610', '11611', '11612', '11613', '11614', '11615')
AND agep BETWEEN 22 AND 24
AND esr != '1'
AND esr != '4'
AND sch = '1'
""",
            conn)

In [132]:

print (sum(ay_df['pwgtp']))
print (sum(oy_df['pwgtp']))

oy_total = sum(oy_df['pwgtp']) #used in racial calculations later

overall_OY_rate = round((sum(oy_df['pwgtp'])) / (sum(ay_df['pwgtp'])) * 100)

overall_OY_rate
print (f'the rate of OY across the whole youth population is {overall_OY_rate}%')

85883.0
11115.0
the rate of OY across the whole youth population is 13%


In [133]:
white_ay = (sum(ay_df[ay_df['rac1p'] == '1']['pwgtp']))
white_oy = (sum(oy_df[oy_df['rac1p'] == '1']['pwgtp']))

white_oy_rate = round((white_oy / white_ay * 100))
white_oy_rate

white_prop_oy = round((white_oy / oy_total*100))
print (white_oy_rate)
print (white_pct_oy_total)

12
50


In [134]:
african_american_ay = (sum(ay_df[ay_df['rac1p'] == '2']['pwgtp']))
african_american_oy = (sum(oy_df[oy_df['rac1p'] == '2']['pwgtp']))

african_american_oy_rate = round((african_american_oy / african_american_ay * 100))
african_american_oy_rate

african_american_prop_oy = round((african_american_oy / oy_total*100))

In [135]:
native_american_ay = sum(ay_df[ay_df['rac1p'] == '2']['pwgtp']) + sum(ay_df[ay_df['rac1p'] == '3']['pwgtp']) + sum(ay_df[ay_df['rac1p'] == '4']['pwgtp'])
native_american_oy = sum(oy_df[oy_df['rac1p'] == '2']['pwgtp']) + sum(oy_df[oy_df['rac1p'] == '3']['pwgtp']) + sum(oy_df[oy_df['rac1p'] == '4']['pwgtp'])

native_american_oy_rate = round((native_american_oy / native_american_ay * 100))
native_american_oy_rate

native_american_prop_oy = round((native_american_oy / oy_total*100))

In [136]:
asian_ay = (sum(ay_df[ay_df['rac1p'] == '6']['pwgtp']))
asian_oy = (sum(oy_df[oy_df['rac1p'] == '6']['pwgtp']))

asian_oy_rate = round((asian_oy / asian_ay * 100))
asian_oy_rate

asian_prop_oy = round((asian_oy / oy_total*100))

In [137]:
pac_islander_ay = (sum(ay_df[ay_df['rac1p'] == '7']['pwgtp']))
pac_islander_oy = (sum(oy_df[oy_df['rac1p'] == '7']['pwgtp']))

pac_islander_oy_rate = round((pac_islander_oy / pac_islander_ay * 100))
pac_islander_oy_rate

pac_islander_prop_oy = round((pac_islander_oy / oy_total*100))

In [138]:
other_race_ay = (sum(ay_df[ay_df['rac1p'] == '8']['pwgtp']))
other_race_oy = (sum(oy_df[oy_df['rac1p'] == '8']['pwgtp']))

other_race_oy_rate = round((other_race_oy / other_race_ay * 100))
other_race_oy_rate

other_race_prop_oy = round((other_race_oy / oy_total*100))

In [139]:
multi_racial_ay = (sum(ay_df[ay_df['rac1p'] == '9']['pwgtp']))
multi_racial_oy = (sum(oy_df[oy_df['rac1p'] == '9']['pwgtp']))

multi_racial_oy_rate = round((multi_racial_oy / multi_racial_ay * 100))
multi_racial_oy_rate

multi_racial_prop_oy = round((multi_racial_oy / oy_total*100))

In [140]:
hispanic_ay = (sum(ay_df[ay_df['anc1p'] == '290']['pwgtp']))
hispanic_oy = (sum(oy_df[oy_df['anc1p'] == '290']['pwgtp']))

hispanic_oy_rate = round((hispanic_oy / hispanic_ay * 100))
hispanic_oy_rate

hispanic_prop_oy = round((hispanic_oy / oy_total*100))

#hispanic did not exist as a category in the rac1p, and we don't know what criterion the other reports used
#so we calculated our hispanic pop based on anc1p = '290' i.e. Hispanic
#this resulted in a sum of propotions higher than 100%
#because we don't know how the groups were tabulated, we have no way of knowing if our data is even relevant 
#when comparing with previous studies
#which leads me to my next slide --- https://en.wikipedia.org/wiki/Radical_transparency

In [149]:
race = ['White', 'African American', 'Native American or Alaska Native', 'Pacific Islander', 'Asian', 'Other Race', 'Two or More Races', 'Hispanic']
ay_total_by_race = [white_ay, african_american_ay, native_american_ay, pac_islander_ay, asian_ay, other_race_ay, multi_racial_ay, hispanic_ay]
oy_total_by_race = [white_oy, african_american_oy, native_american_oy, pac_islander_oy, asian_oy, other_race_oy, multi_racial_oy, hispanic_oy]
racial_rate_of_oy = [white_oy_rate, african_american_oy_rate, native_american_oy_rate, pac_islander_oy_rate, asian_oy_rate, other_race_oy_rate, multi_racial_oy_rate, hispanic_oy_rate]
racial_proportion_of_oy = [white_prop_oy,african_american_prop_oy, native_american_prop_oy, pac_islander_prop_oy, asian_prop_oy, other_race_prop_oy, multi_racial_prop_oy, hispanic_prop_oy ]
racial_df_columns = ['Race', 'Rate of OY', 'Total Youth Populaion', 'Proportion of OY', 'OY Total']
racial_df_data = [race, oy_total_by_race, oy_pct_by_race]

racial_df = pd.DataFrame(race)

In [150]:
racial_df['a'] = racial_rate_of_oy
racial_df['b'] = ay_total_by_race
racial_df['c'] = racial_proportion_of_oy
racial_df['d'] = oy_total_by_race

In [151]:
racial_df.columns = racial_df_columns

In [152]:
racial_df = racial_df.sort_values(by=['Rate of OY'], ascending=False)

In [153]:
racial_df

,Race,Rate of OY,Total Youth Populaion,Proportion of OY,OY Total
7,Hispanic,36,355.0,1,128.0
3,Pacific Islander,20,1877.0,3,373.0
2,Native American or Alaska Native,17,9509.0,14,1579.0
6,Two or More Races,16,7836.0,11,1257.0
1,African American,15,8920.0,12,1343.0
5,Other Race,14,7298.0,9,1052.0
0,White,12,45663.0,50,5547.0
4,Asian,9,13328.0,11,1196.0


In [154]:
racial_df.to_csv('./racial_data.csv')

In [155]:
ls

01_erh_download_and_explore_data.ipynb  README.md
02_samt_exploratory.ipynb               project_outcomes.ipynb
Kyle_Exploration.ipynb                  racial_data.csv


In [156]:
racial_df['Proportion of OY'].sum()

111